Imports

In [2]:
import pandas as pd
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.applications import ResNet50

2024-10-09 17:31:27.295613: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 17:31:27.328623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 17:31:27.902674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Paths to the dataset. Loading the Datasets

In [3]:
# Paths to the dataset
train_image_folder = '../COMP90086_2024_Project_train/train'
metadata_file = '../COMP90086_2024_Project_train/train.csv'

# Load the metadata CSV file
metadata = pd.read_csv(metadata_file)

Load the dataset function

In [4]:
def load_images_and_metadata(image_folder, metadata_df):
    images = []
    metadata_features = []
    labels = []
    
    for idx, row in metadata_df.iterrows():
        # Construct the image path
        img_path = os.path.join(image_folder, f'{row["id"]}.jpg')
        
        # Check if the image exists
        if not os.path.exists(img_path):
            print(f"Image {img_path} not found.")
            continue
        
        # Load the image
        image = cv2.imread(img_path)
        
        # Check if the image is loaded correctly
        if image is None:
            print(f"Failed to load image {img_path}")
            continue
        
        # Resize the image
        image = cv2.resize(image, (224, 224))  # Resize to a fixed size
        image = img_to_array(image) / 255.0  # Normalize image
        
        # Append the image and metadata
        images.append(image)
        
        # Select relevant metadata features (e.g., shapeset, type, total_height, etc.)
        metadata_features.append([row['shapeset'], row['type'], row['total_height']])
        
        # Append the label (stable_height)
        labels.append(row['stable_height'])
    
    return np.array(images), np.array(metadata_features), np.array(labels)


Load and split dataset for training

In [5]:
# Load training data
images, metadata_features, labels = load_images_and_metadata(train_image_folder, metadata)

# Split the data into training and validation sets
X_train_img, X_val_img, X_train_meta, X_val_meta, y_train, y_val = train_test_split(
    images, metadata_features, labels, test_size=0.2, random_state=42)

Creation of the model

In [6]:
# Image model branch
input_img = Input(shape=(224, 224, 3), name='image_input')
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model
x = base_model(input_img)
x = Flatten()(x)

# Metadata model branch
input_meta = Input(shape=(3,), name='metadata_input')
meta = Dense(32, activation='relu')(input_meta)

# Concatenate image and metadata branches
combined = Concatenate()([x, meta])
combined = Dense(128, activation='relu')(combined)
combined = Dense(64, activation='relu')(combined)
output = Dense(1, name='output')(combined)  # Regression output for stable height

# Compile the model
model = Model(inputs=[input_img, input_meta], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Display the model architecture
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ image_input[0][0] │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ metadata_input      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 100352)    │          0 │ resnet50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        128 │ metadata_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 100384)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │ 12,849,280 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 36,445,441 (139.03 MB)

 Trainable params: 12,857,729 (49.05 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Train and save the model

In [7]:
# Train the model
history = model.fit(
    [X_train_img, X_train_meta], y_train,
    validation_data=([X_val_img, X_val_meta], y_val),
    epochs=20,  # Adjust epochs as needed
    batch_size=32,
    verbose=1
)

# Save the trained model
model.save('model.h5')

Epoch 1/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 190s 970ms/step - loss: 31.2209 - mae: 2.6493 - val_loss: 2.1884 - val_mae: 1.2446
Epoch 2/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 186s 970ms/step - loss: 2.1520 - mae: 1.2161 - val_loss: 2.0933 - val_mae: 1.2043
Epoch 3/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 187s 972ms/step - loss: 2.0164 - mae: 1.1773 - val_loss: 3.1821 - val_mae: 1.3940
Epoch 4/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 186s 971ms/step - loss: 2.3379 - mae: 1.2470 - val_loss: 2.0759 - val_mae: 1.2177
Epoch 5/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 186s 971ms/step - loss: 1.9588 - mae: 1.1502 - val_loss: 2.1230 - val_mae: 1.2291
Epoch 6/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 186s 969ms/step - loss: 1.9138 - mae: 1.1416 - val_loss: 1.9384 - val_mae: 1.1624
Epoch 7/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 186s 968ms/step - loss: 1.7692 - mae: 1.0939 - val_loss: 2.0059 - val_mae: 1.1577
Epoch 8/20
192/192 ━━━━━━━━━━━━━━━━━━━━ 187s 972ms/step - loss: 1.7942 - mae: 1.0987 - val_loss: 1.8381 - val_mae: 1.1321
Epoch 9/20
192/192 ━━━━

ValueError: Unable to synchronously create dataset (name already exists)

In [9]:
model.save('hgchkgc.h5')

ValueError: Unable to synchronously create dataset (name already exists)

Model evaluation

In [10]:
# Evaluate the model
val_loss, val_mae = model.evaluate([X_val_img, X_val_meta], y_val)
print(f'Validation Loss: {val_loss}, Validation MAE: {val_mae}')

48/48 ━━━━━━━━━━━━━━━━━━━━ 35s 721ms/step - loss: 1.8934 - mae: 1.1327
Validation Loss: 1.8776227235794067, Validation MAE: 1.136664867401123


Make the predictions

In [13]:
# Load test images and metadata similarly to training
# Assuming test images and metadata are in similar format
test_image_folder = '../COMP90086_2024_Project_test/test'
test_metadata_file = '../COMP90086_2024_Project_test/test.csv'
test_metadata = pd.read_csv(test_metadata_file)

# Load the test data
test_images, test_metadata_features, _ = load_images_and_metadata(test_image_folder, test_metadata)

# Run predictions
predictions = model.predict([test_images, test_metadata_features])

# Save predictions in the required format
submission = pd.DataFrame({
    'id': test_metadata['id'],
    'stable_height': predictions.flatten().astype(int)
})
submission.to_csv('predictions.csv', index=False)

KeyError: 'shapeset'